<a href="https://colab.research.google.com/github/nanderson-rodrigues/projeto-etl/blob/main/projeto_etl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pandera

In [2]:
import pandas as pd
import pandera as pa

In [35]:
df = pd.read_csv("ocorrencia_2010_2020.csv", parse_dates=['ocorrencia_dia'], dayfirst=True)
df.head(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
5,39807,39807,INCIDENTE,SALVADOR,BA,****,2010-01-06,17:53:00,0
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,****,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,****,2010-01-10,20:00:00,0


### Validação

In [36]:
schema = pa.DataFrameSchema(
    columns = {
        "codigo_ocorrencia": pa.Column(pa.Int),
        "codigo_ocorrencia2": pa.Column(pa.Int),
        "ocorrencia_classificacao": pa.Column(pa.String),
        "ocorrencia_cidade": pa.Column(pa.String),
        "ocorrencia_uf": pa.Column(pa.String, pa.Check.str_length(2,2)),
        "ocorrencia_aerodromo": pa.Column(pa.String),
        "ocorrencia_dia": pa.Column(pa.DateTime),
        "ocorrencia_hora": pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True),
        "total_recomendacoes": pa.Column(pa.Int)
    }
)

In [37]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5735,79769,79769,INCIDENTE GRAVE,MANOEL URBANO,AC,SIMB,2020-12-29,18:30:00,0
5736,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5737,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5738,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0


In [ ]:
df.dtypes

codigo_ocorrencia                    int64
codigo_ocorrencia2                   int64
ocorrencia_classificacao            object
ocorrencia_cidade                   object
ocorrencia_uf                       object
ocorrencia_aerodromo                object
ocorrencia_dia              datetime64[ns]
ocorrencia_hora                     object
total_recomendacoes                  int64
dtype: object

In [ ]:
df.ocorrencia_dia.dt.month

0        1
1        1
2        1
3        1
4        1
        ..
5735    12
5736    12
5737    12
5738    12
5739    12
Name: ocorrencia_dia, Length: 5740, dtype: int64

In [ ]:
df.loc[:,'ocorrencia_cidade']

0           RIO DE JANEIRO
1                    BELÉM
2           RIO DE JANEIRO
3       LUCAS DO RIO VERDE
4                  PELOTAS
               ...        
5735         MANOEL URBANO
5736              CAMPINAS
5737     LAGOA DA CONFUSÃO
5738        RIO DE JANEIRO
5739             VICENTINA
Name: ocorrencia_cidade, Length: 5740, dtype: object

In [ ]:
df.codigo_ocorrencia.is_unique

True

###Limpeza


In [38]:
df.loc[df.ocorrencia_aerodromo == '****', ['ocorrencia_aerodromo']] = pd.NA

In [39]:
df.head()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0


In [40]:
df.replace(['**','###!', '####', '****', '*****', 'NULL'], pd.NA, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/missing.py:49: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask = arr == x
/usr/local/lib/python3.7/dist-packages/pandas/core/missing.py:60: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= arr == x


In [ ]:
df.head(100)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,<NA>,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,<NA>,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
95,40329,40329,INCIDENTE,SÃO PAULO,SP,SBSP,2010-03-18,12:10:00,0
96,41998,41998,INCIDENTE,CONFINS,MG,SBCF,2010-03-18,13:37:00,0
97,41999,41999,INCIDENTE,GUARULHOS,SP,SBGR,2010-03-18,03:00:00,0
98,40088,40088,ACIDENTE,UIRAMUTÃ,RR,<NA>,2010-03-19,13:10:00,3


In [ ]:
df.isna()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,True,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
5735,False,False,False,False,False,False,False,False,False
5736,False,False,False,False,False,False,False,False,False
5737,False,False,False,False,False,True,False,False,False
5738,False,False,False,False,False,False,False,False,False


In [ ]:
df.isna().sum()

codigo_ocorrencia              0
codigo_ocorrencia2             0
ocorrencia_classificacao       0
ocorrencia_cidade              0
ocorrencia_uf                  0
ocorrencia_aerodromo        2176
ocorrencia_dia                 0
ocorrencia_hora                1
total_recomendacoes            0
dtype: int64

### Transformação


- Limpando de outra forma:

In [41]:
valores_ausentes = ['**','###!', '####', '****', '*****', 'NULL']
df = pd.read_csv("ocorrencia_2010_2020.csv", parse_dates=['ocorrencia_dia'], dayfirst=True, na_values=valores_ausentes)
df.head(10)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
5,39807,39807,INCIDENTE,SALVADOR,BA,NaN,2010-01-06,17:53:00,0
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0


In [42]:
schema = pa.DataFrameSchema(
    columns = {
        "codigo_ocorrencia": pa.Column(pa.Int),
        "codigo_ocorrencia2": pa.Column(pa.Int),
        "ocorrencia_classificacao": pa.Column(pa.String),
        "ocorrencia_cidade": pa.Column(pa.String),
        "ocorrencia_uf": pa.Column(pa.String, pa.Check.str_length(2,2), nullable=True),
        "ocorrencia_aerodromo": pa.Column(pa.String, nullable=True),
        "ocorrencia_dia": pa.Column(pa.DateTime),
        "ocorrencia_hora": pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9])(:[0-5][0-9])?$'), nullable=True),
        "total_recomendacoes": pa.Column(pa.Int)
    }
)

In [43]:
schema.validate(df)

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,NaN,2010-01-04,17:30:00,0
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0
...,...,...,...,...,...,...,...,...,...
5735,79769,79769,INCIDENTE GRAVE,MANOEL URBANO,AC,SIMB,2020-12-29,18:30:00,0
5736,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5737,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,NaN,2020-12-30,18:30:00,0
5738,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0


In [11]:
#ocorrencias onde aja incidentes graves ou não em SP
filtro1 = df.ocorrencia_classificacao.isin(['INCIDENTE GRAVE', 'INCIDENTE'])
filtro2 = df.ocorrencia_uf == 'SP'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0
18,39828,39828,INCIDENTE,SANTOS,SP,NaN,2010-01-15,17:45:00,0
26,39847,39847,INCIDENTE,SOROCABA,SP,NaN,2010-01-20,13:10:00,0
27,39768,39768,INCIDENTE,CAMPINAS,SP,NaN,2010-01-21,20:45:00,0
33,39848,39848,INCIDENTE,GUARULHOS,SP,SBGR,2010-01-25,17:05:00,0
...,...,...,...,...,...,...,...,...,...
5694,79739,79739,INCIDENTE GRAVE,SÃO PAULO,SP,SBSP,2020-12-03,14:42:00,0
5695,79705,79705,INCIDENTE GRAVE,SOROCABA,SP,SDCO,2020-12-04,15:30:00,0
5705,79718,79718,INCIDENTE GRAVE,SÃO PAULO,SP,SBMT,2020-12-11,13:50:00,0
5731,79800,79800,INCIDENTE,SÃO PAULO,SP,SBMT,2020-12-28,10:15:00,0


In [13]:
#ocorrencia onde cidades comecem com a letra C
filtro = df.ocorrencia_cidade.str[0] == 'C'
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
6,40215,40215,INCIDENTE,COARI,AM,SBUY,2010-01-07,18:40:00,0
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
8,39156,39156,INCIDENTE GRAVE,CASCAVEL,PR,SBCA,2010-01-10,23:15:00,2
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
20,40310,40310,INCIDENTE,CRUZEIRO DO SUL,AC,SBCZ,2010-01-18,19:32:00,0
...,...,...,...,...,...,...,...,...,...
5652,79649,79649,INCIDENTE,CAMPINAS,SP,SBKP,2020-11-04,22:39:00,0
5654,79652,79652,INCIDENTE,CONFINS,MG,SBCF,2020-11-06,14:20:00,0
5670,79683,79683,INCIDENTE,CRUZEIRO DO SUL,AC,SBCZ,2020-11-16,22:00:00,0
5686,79692,79692,ACIDENTE,CATANDUVA,SP,SDCD,2020-11-28,12:50:00,0


In [14]:
#ocorrencia onde cidades terminam com a letra A
filtro = df.ocorrencia_cidade.str[-1] == 'A'
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
12,40414,40414,INCIDENTE,BRASÍLIA,DF,SBBR,2010-01-10,03:00:00,0
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
25,39772,39772,INCIDENTE,UBERLÂNDIA,MG,NaN,2010-01-20,18:58:00,0
26,39847,39847,INCIDENTE,SOROCABA,SP,NaN,2010-01-20,13:10:00,0
...,...,...,...,...,...,...,...,...,...
5727,79787,79787,INCIDENTE,FORTALEZA,CE,SBFZ,2020-12-23,18:30:00,0
5728,79749,79749,ACIDENTE,MARITUBA,PA,NaN,2020-12-26,12:05:00,0
5732,79824,79824,ACIDENTE,RIO PARANAÍBA,MG,SNRP,2020-12-28,17:00:00,0
5733,79753,79753,INCIDENTE GRAVE,GOIATUBA,GO,NaN,2020-12-29,12:00:00,0


In [15]:
#ocorrencia onde cidades terminam com MA
filtro = df.ocorrencia_cidade.str[-2:] == 'MA'
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
408,43062,43062,ACIDENTE,NOVO GAMA,GO,NaN,2010-09-25,16:30:00,1
688,44198,44198,ACIDENTE,BURITAMA,SP,NaN,2011-03-19,11:18:00,0
1035,51785,51785,INCIDENTE,BURITAMA,SP,SIBX,2011-10-25,19:15:00,0
1377,45939,45939,ACIDENTE,UMUARAMA,PR,NaN,2012-05-30,19:00:00,0
1401,46180,46180,ACIDENTE,AURIFLAMA,SP,NaN,2012-06-16,19:30:00,0
2099,49113,49113,INCIDENTE GRAVE,NOVA LIMA,MG,NaN,2013-06-15,12:00:00,4
2102,48799,48799,ACIDENTE,CANUTAMA,AM,NaN,2013-06-18,21:30:00,0
2655,51347,51347,INCIDENTE,DIADEMA,SP,NaN,2014-05-04,16:00:00,0


In [16]:
#ocorrencia onde nomes de cidades contêm MA
filtro = df.ocorrencia_cidade.str.contains('MA')
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
7,39707,39707,INCIDENTE GRAVE,CANUTAMA,AM,NaN,2010-01-09,12:30:00,3
13,39507,39507,ACIDENTE,PRIMAVERA DO LESTE,MT,NaN,2010-01-11,17:00:00,2
15,39315,39315,ACIDENTE,CANUTAMA,AM,NaN,2010-01-15,21:22:00,0
35,39487,39487,INCIDENTE GRAVE,PALMAS,TO,SBPJ,2010-01-26,16:50:00,0
78,39916,39916,INCIDENTE GRAVE,MANAUS,AM,SBEG,2010-03-07,19:35:00,2
...,...,...,...,...,...,...,...,...,...
5722,79745,79745,INCIDENTE,MARABÁ,PA,SBMA,2020-12-21,08:47:00,0
5728,79749,79749,ACIDENTE,MARITUBA,PA,NaN,2020-12-26,12:05:00,0
5730,79754,79754,INCIDENTE GRAVE,NOVA MARINGÁ,MT,NaN,2020-12-28,13:00:00,0
5734,79755,79755,ACIDENTE,MATO RICO,PR,NaN,2020-12-29,10:30:00,0


In [27]:
filtro = df.ocorrencia_dia.dt.year == 2020
df.loc[filtro]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes
5237,79043,79043,ACIDENTE,CAMANDUCAIA,MG,SNEJ,2020-01-01,21:00:00,0
5238,79044,79044,INCIDENTE GRAVE,SALVADOR,BA,SBSV,2020-01-03,16:35:00,0
5239,79049,79049,ACIDENTE,LEME,SP,****,2020-01-03,21:15:00,0
5240,79132,79132,INCIDENTE,SÃO MATEUS,ES,****,2020-01-03,14:10:00,0
5241,79182,79182,INCIDENTE,GOIÂNIA,GO,SBGO,2020-01-03,11:10:00,0
...,...,...,...,...,...,...,...,...,...
5735,79769,79769,INCIDENTE GRAVE,MANOEL URBANO,AC,SIMB,2020-12-29,18:30:00,0
5736,79804,79804,INCIDENTE,CAMPINAS,SP,SBKP,2020-12-29,19:00:00,0
5737,79757,79757,INCIDENTE GRAVE,LAGOA DA CONFUSÃO,TO,****,2020-12-30,18:30:00,0
5738,79802,79802,INCIDENTE,RIO DE JANEIRO,RJ,SBGL,2020-12-30,00:54:00,0


In [45]:
#juntando coluna de data e hora e preservando o tipo datetime
df['ocorrencia_dia_hora'] = pd.to_datetime( df.ocorrencia_dia.astype(str) + ' ' + df.ocorrencia_hora )

In [30]:
df.head()

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,****,2010-01-03,12:00:00,0,2010-01-03 12:00:00
1,40349,40349,INCIDENTE,BELÉM,PA,SBBE,2010-01-03,11:05:00,0,2010-01-03 11:05:00
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,2010-01-03 03:00:00
3,39527,39527,ACIDENTE,LUCAS DO RIO VERDE,MT,****,2010-01-04,17:30:00,0,2010-01-04 17:30:00
4,40324,40324,INCIDENTE,PELOTAS,RS,SBPK,2010-01-05,19:25:00,0,2010-01-05 19:25:00


In [31]:
filtro1 = df.ocorrencia_dia_hora >= '2015-12-03 11:00:00'
filtro2 = df.ocorrencia_dia_hora <=  '2015-12-08 11:00:00'
df.loc[filtro1 & filtro2]

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
3431,60637,60637,INCIDENTE,BELO HORIZONTE,MG,SBBH,2015-12-03,16:47:00,0,2015-12-03 16:47:00
3432,53625,53625,ACIDENTE,TRINDADE,GO,****,2015-12-06,13:10:00,3,2015-12-06 13:10:00
3433,53626,53626,ACIDENTE,AMERICANA,SP,SDAI,2015-12-06,15:00:00,1,2015-12-06 15:00:00
3435,53629,53629,ACIDENTE,JALES,SP,SDJL,2015-12-08,10:20:00,0,2015-12-08 10:20:00


In [46]:
#criando dataframe para o período de 03-2015
filtro1 = df.ocorrencia_dia_hora.dt.year == 2015
filtro2 = df.ocorrencia_dia_hora.dt.month == 3 
df201503 = df.loc[filtro1 & filtro2]
df201503

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
3084,53183,53183,ACIDENTE,AMAPORÃ,PR,NaN,2015-03-02,23:00:00,0,2015-03-02 23:00:00
3085,53120,53120,ACIDENTE,CHAVES,PA,SNXW,2015-03-04,13:30:00,0,2015-03-04 13:30:00
3086,53109,53109,ACIDENTE,CAMPO GRANDE,MS,SSIE,2015-03-05,13:50:00,0,2015-03-05 13:50:00
3087,53112,53112,ACIDENTE,MOGI GUAÇU,SP,NaN,2015-03-06,21:00:00,0,2015-03-06 21:00:00
3088,53152,53152,INCIDENTE GRAVE,RIO DE JANEIRO,RJ,SBRJ,2015-03-10,11:30:00,2,2015-03-10 11:30:00
3089,53167,53167,INCIDENTE,MARABÁ,PA,SBMA,2015-03-10,17:33:00,0,2015-03-10 17:33:00
3090,53596,53596,INCIDENTE,ITAPEMA,SC,NaN,2015-03-10,21:35:00,0,2015-03-10 21:35:00
3091,53149,53149,ACIDENTE,TABATINGA,AM,SBTT,2015-03-11,23:25:00,0,2015-03-11 23:25:00
3092,53148,53148,ACIDENTE,ARARAS,SP,SDEH,2015-03-12,12:40:00,0,2015-03-12 12:40:00
3093,53153,53153,INCIDENTE,PORTO ALEGRE,RS,SBPA,2015-03-13,15:45:00,0,2015-03-13 15:45:00


In [47]:
df201503.count()

codigo_ocorrencia           37
codigo_ocorrencia2          37
ocorrencia_classificacao    37
ocorrencia_cidade           37
ocorrencia_uf               37
ocorrencia_aerodromo        21
ocorrencia_dia              37
ocorrencia_hora             37
total_recomendacoes         37
ocorrencia_dia_hora         37
dtype: int64

In [49]:
df201503.groupby(['ocorrencia_classificacao']).codigo_ocorrencia.count()

ocorrencia_classificacao
ACIDENTE           15
INCIDENTE          17
INCIDENTE GRAVE     5
Name: codigo_ocorrencia, dtype: int64

In [50]:
#ao contar usando a ocorrencia_aerodromo a contagem sai 'errada' pois há valores nulos em alguns lugares
df201503.groupby(['ocorrencia_classificacao']).ocorrencia_aerodromo.count()

ocorrencia_classificacao
ACIDENTE            5
INCIDENTE          14
INCIDENTE GRAVE     2
Name: ocorrencia_aerodromo, dtype: int64

In [51]:
#melhor utilizar size() já que ele conta os registros
df201503.groupby(['ocorrencia_classificacao']).size()

ocorrencia_classificacao
ACIDENTE           15
INCIDENTE          17
INCIDENTE GRAVE     5
dtype: int64

In [53]:
 filtro1 = df.ocorrencia_dia.dt.year == 2010
 filtro2 = df.ocorrencia_uf.isin(['SP', 'MG', 'ES', 'RJ'])
 dfsudeste2010 = df.loc[filtro1 & filtro2]
 dfsudeste2010

,codigo_ocorrencia,codigo_ocorrencia2,ocorrencia_classificacao,ocorrencia_cidade,ocorrencia_uf,ocorrencia_aerodromo,ocorrencia_dia,ocorrencia_hora,total_recomendacoes,ocorrencia_dia_hora
0,40211,40211,INCIDENTE,RIO DE JANEIRO,RJ,NaN,2010-01-03,12:00:00,0,2010-01-03 12:00:00
2,40351,40351,INCIDENTE,RIO DE JANEIRO,RJ,SBRJ,2010-01-03,03:00:00,0,2010-01-03 03:00:00
9,39711,39711,INCIDENTE GRAVE,PARÁ DE MINAS,MG,NaN,2010-01-10,20:00:00,0,2010-01-10 20:00:00
11,40069,40069,ACIDENTE,SÃO PAULO,SP,SBMT,2010-01-10,14:50:00,8,2010-01-10 14:50:00
16,39809,39809,INCIDENTE,SÃO PAULO,SP,NaN,2010-01-15,15:00:00,0,2010-01-15 15:00:00
...,...,...,...,...,...,...,...,...,...,...
541,43722,43722,ACIDENTE,CAJOBI,SP,NaN,2010-12-23,11:00:00,0,2010-12-23 11:00:00
544,43548,43548,INCIDENTE,SÃO PAULO,SP,SBSP,2010-12-26,22:10:00,0,2010-12-26 22:10:00
548,43724,43724,ACIDENTE,COSMÓPOLIS,SP,NaN,2010-12-29,12:00:00,0,2010-12-29 12:00:00
550,43730,43730,INCIDENTE,VITÓRIA,ES,NaN,2010-12-30,18:10:00,0,2010-12-30 18:10:00


In [54]:
dfsudeste2010.groupby(['ocorrencia_classificacao']).size()

ocorrencia_classificacao
ACIDENTE            40
INCIDENTE          170
INCIDENTE GRAVE     18
dtype: int64

In [55]:
dfsudeste2010.groupby(['ocorrencia_classificacao', 'ocorrencia_uf']).size()

ocorrencia_classificacao  ocorrencia_uf
ACIDENTE                  MG                7
                          RJ                9
                          SP               24
INCIDENTE                 ES                5
                          MG               39
                          RJ               52
                          SP               74
INCIDENTE GRAVE           MG                5
                          RJ                4
                          SP                9
dtype: int64

In [56]:
dfsudeste2010.groupby(['ocorrencia_cidade']).size().sort_values(ascending=False)

ocorrencia_cidade
RIO DE JANEIRO           47
SÃO PAULO                33
GUARULHOS                18
BELO HORIZONTE           17
CAMPINAS                 13
MACAÉ                    12
CONFINS                   7
UBERLÂNDIA                5
VITÓRIA                   4
JUIZ DE FORA              4
SÃO JOSÉ DO RIO PRETO     3
SÃO JOSÉ DOS CAMPOS       3
MARÍLIA                   3
ARARAQUARA                2
MONTES CLAROS             2
CAMPOS DOS GOYTACAZES     2
DIVINÓPOLIS               2
IPATINGA                  2
ITANHAÉM                  2
JUNDIAÍ                   2
POÇOS DE CALDAS           2
RIBEIRÃO PRETO            2
SOROCABA                  2
UBERABA                   2
BAURU                     2
AREALVA                   2
BOITUVA                   1
BOCAINA                   1
BOM JESUS DO AMPARO       1
BOM JESUS DO GALHO        1
BEBEDOURO                 1
BRAGANÇA PAULISTA         1
BROTAS                    1
CAJOBI                    1
ARRAIAL DO CABO           1
CA

In [58]:
dfsudeste2010.groupby(['ocorrencia_aerodromo'], dropna=False).total_recomendacoes.sum()

ocorrencia_aerodromo
SBAE     0
SBAQ     0
SBBH     0
SBBP     9
SBBU     0
SBCF     0
SBGL     3
SBGR     0
SBIP     0
SBJD     7
SBJR     3
SBKP     0
SBME     0
SBMK     0
SBML     0
SBMT    27
SBPC     0
SBPR     1
SBRJ    19
SBRP     0
SBSJ     0
SBSP     0
SBSR     0
SBUL     0
SBUR     0
SBVT     0
SDAM     0
SDOI     1
SDPW     0
SDPY     0
SNDT     0
SNDV     0
SNXT     0
NaN     45
Name: total_recomendacoes, dtype: int64